In [1]:
#pip install polytope


In [2]:
#pip install pypoman

In [3]:
# pip install polytope
# pip install pypoman

import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
from pypoman import compute_polytope_halfspaces, compute_polytope_vertices
from pypoman.duality import convex_hull
from polytope import polytope
from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull

Let's construct a reflexive 2D polytope (taken from the book "The Calabi–Yau Landscape", page 44)

In [4]:
V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])


Pypoman provides a form $0\le -Ax+b$

In [5]:
A, b = compute_polytope_halfspaces(V)

print(A)
print(b)

#print(compute_polytope_vertices(A, b))
print(convex_hull(V))

[[-0. -1.]
 [-1. -0.]
 [-1. -1.]
 [ 1.  1.]
 [ 1. -0.]]
[1. 1. 1. 1. 1.]
[array([-1,  2]), array([-1,  0]), array([ 0, -1]), array([ 1, -1]), array([1, 0])]


Polytope provides a form $Ax\le b$ which is equivalent to pypoman's form $0\le-Ax+b$

In [6]:
# P = qhull(V)

# print(P.A)
# print(P.b)
# #print(P.vertices)

Scipy yields an array $[normal, offset]$ that describes $Ax+b\le0$ forming the hyperplane equation of the facet see [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.ConvexHull.html)

In [7]:
# hull = ConvexHull(V)
# print(hull.equations)

When using the negated form of pypoman's result, that is $Ax-b\le0$ then $b$ is the distance we are looking for.

The distances are defined in the hyperplane representation. The hyperplanes are given as $<u,v> \ge a$ where $u$ are points in the polytope lattice $M, v$ is a single point in the dual lattice $N$ and $a$ is the distance which is real number. If we just get the hyperplane representation then we automatically have the distances.

In [8]:
# A, b = compute_polytope_halfspaces(V)
# print(b)

Count integral points using polytope library.
An interesting reference is also [Effective lattice point counting in rational convex polytopes](https://www.sciencedirect.com/science/article/pii/S0747717104000422?via%3Dihub)

In [9]:
# V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
# polytope.ABS_TOL = 0
# poly_qhull = qhull(V)
# print(poly_qhull)
# integral_points = enumerate_integral_points(poly_qhull)
# integral_points = integral_points.transpose()
# integral_points = integral_points.astype(int)
# print(integral_points)

The fitness function consists of two parts.

The first term is IP(Delta)-1, where IP(Delta) = 1 if Delta satisfies IP and 0 otherwise, adds a penalty if Delta doesn’t satisfy the IP property.

The second term sum(ai-1), where ai are the hyperplane distances of each face, adds a penalty if the distances of all the hyper planes aren’t at a distance 1 from the origin.

Note: Ensure that when you generate polytopes that you choose its points around the origin! Otherwise you have to translate the polytope such that its origin or one of its interior points becomes $(0,0,0,\ldots)$.

In [10]:
# #
# # 2D Example
# #
# # V = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
# V = np.array([[1, 3], [1, 1], [1, 2], [1, 1], [3, 1]])

# #
# # 3D Example
# #
# #good cube: its origin is (1,1,1)
# # V = np.array([[0,0,0], [2,0,0], [2,2,0], [0,2,0], [0,2,2], [2,2,2], [2,0,2], [0,0,2]])

# #good cube: its origin is (0,0,0)
# #
# # 5D Example
# #
# # V = np.array([[-1,0,1,1,-1], [0,1,-1,-1,1], [0,-3,-1,1,0], [-1,0,0,-1,0], [-1,1,-1,-1,1], [0,-1,1,1,0], [0,0,0,-1,0]])
# # V = np.array([[-2,0,1,0,-1], [1,0,1,-1,2], [-1,1,-1,-2,0], [1,-1,0,2,0], [1,0,0,-1,0], [0,0,0,2,-1]])

# A, b = compute_polytope_halfspaces(V)

# poly_qhull = qhull(V)
# print(poly_qhull)

# def grid_region(polyreg, res=None):
#     # grid corners
#     bbox = polyreg.bounding_box
#     # grid resolution
#     if res is None:
#         density = 8
#         res = [
#             math.ceil(density * (b - a))
#             for a, b in zip(*bbox)]
#     if len(res) != polyreg.dim:
#         raise ValueError((
#             "`len(res)` must equal the polytope's dimension "
#             "(which is {dim}), but instead `res` is:  {res}"
#             ).format(dim=polyreg.dim, res=res))
#     if any(n < 1 for n in res):
#         raise ValueError((
#             '`res` must contain `int` values >= 1, '
#             'instead `res` equals:  {res}'
#             ).format(res=res))
#     linspaces = list()
#     for a, b, n in zip(*bbox, res):
#         r = np.linspace(a, b, num=n)
#         linspaces.append(r)
#     points = np.meshgrid(*linspaces)
#     x = np.vstack(list(map(np.ravel, points)))
#     x = x[:, polyreg.contains(x, abs_tol=0)]
#     return (x, res)

# def enumerate_stricktly_integral_points(poly):
#     a, b = poly.bounding_box
#     a_int = np.floor(a)
#     b_int = np.ceil(b)
#     intervals = list(zip(a_int.flatten(), b_int.flatten()))
#     box = box2poly(intervals)
#     res = [int(b - a + 1) for a, b in intervals]
#     grid, _ = grid_region(box, res=res)
#     inside = poly.contains(grid, abs_tol=0)
#     return grid[:, inside]

# integral_points = enumerate_integral_points(poly_qhull)
# integral_points = integral_points.transpose()
# integral_points = integral_points.astype(int)

# stricktly_integral_points = enumerate_stricktly_integral_points(poly_qhull)
# stricktly_integral_points = stricktly_integral_points.transpose()
# stricktly_integral_points = stricktly_integral_points.astype(int)
# num_stricktly_integral_points = len(stricktly_integral_points)

# print("integral_points: {0}".format(integral_points.tolist()))
# print("stricktly_integral_points: {0}".format(stricktly_integral_points.tolist()))
# print("b: {0}".format(b))

# def fitness(ip_count, distances):
#     result = 0
#     if ip_count > 1:
#         result -= 1
#     for d in distances:
#         result -= abs(d-1)
#     return result

# print("fitness value: {0}".format(fitness(num_stricktly_integral_points, b)))

Some experimental stuff starts here:

# GA TEST

In [11]:
import math
import numpy as np
from sympy.geometry import Point
import matplotlib.pyplot as plt
from pypoman import compute_polytope_halfspaces, compute_polytope_vertices
from pypoman.duality import convex_hull
from polytope import polytope
from polytope.polytope import enumerate_integral_points, qhull, box2poly
from scipy.spatial import ConvexHull
import random
from random import randint
import copy



class Population:
    def __init__(self, number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #self.generation_0 = Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        
        self.number_of_generations = number_of_generations
        self.all_generations = []
        #print("yyyyyyyyyyyyyyyyyyyyyyyyy")
        
        
        # try:
        #     self.all_generations.append(Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        # except:
        #     chrom_list = random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions)
        #     self.all_generations.append(Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        
        
        
        
        self.all_generations.append(Generation(chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
        

        
        for curr_generation_number in range(0, number_of_generations-1):
            
            self.copy_prev_Generation_chrom_list = copy.deepcopy(self.all_generations[curr_generation_number].get_chrom_list())
#############################################################################################################################
            # test_generation = Generation(self.copy_prev_Generation_chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
            # fitness_of_test = test_generation.get_fitness_values_list()
            # print(fitness_of_test)
            #############################################################################################################################
            #copy_prev_Generation_chrom_list
            self.copy_prev_Generation_chrom_list = self.copy_prev_Generation_chrom_list[:int(len(self.copy_prev_Generation_chrom_list)/2)]
            #print(copy_prev_Generation_chrom_list)
####################################################################################################################################
            # test_generation = Generation(self.copy_prev_Generation_chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
            # fitness_of_test = test_generation.get_fitness_values_list()
            # print(fitness_of_test)
            #############################################################################################################################

            parents = []
            #for curr_parent_choosing_round in range(0,len(self.all_generations[0])):
            #print(len(self.all_generations[0].get_chrom_list()))
            #print("yyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy")
            while len(self.copy_prev_Generation_chrom_list) < len(self.all_generations[0].get_chrom_list()):

                parents = random.sample(self.copy_prev_Generation_chrom_list, 2)
                

                self.copy_prev_Generation_chrom_list.append(Chromosom.apply_crossover(parents[0], parents[1], crossover_type))
                
            #print(len(copy_prev_Generation_chrom_list))
                
            #if len(copy_generation) < len(self.all_generations[0]
            


            self.copy_prev_Generation_chrom_list.sort(key=lambda x: x.fitness, reverse=True)
            #print(copy_prev_Generation_chrom_list)

            self.all_generations.append(Generation(self.copy_prev_Generation_chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point))
            # print(self.all_generations[len(self.all_generations)-1].get_fitness_values_list())
            # print("-----------------------------------------------")
    def get_all_generations(self):
        return self.all_generations


class Generation:
    num_points_per_polytope = 0
    num_chroms_per_Generation = 0
    num_dimensions = 0 # 3
    min_value_per_point = 0  # -3
    max_value_per_point = 0  # 3
    
    
    def __init__(self, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        
        Generation.num_points_per_polytope = num_points_per_polytope
        Generation.num_chroms_per_Generation = num_chroms_per_Generation
        Generation.num_dimensions = num_dimensions # 3
        Generation.min_value_per_point = min_value_per_point  # -3
        Generation.max_value_per_point = max_value_per_point  # 3
        
        

        if chrom_list == None:
            self.chrom_list = self.gen_first_generation(num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
        else:
            self.chrom_list = chrom_list

        self.generation_fitness_list = self.set_fitness_value_list(self.chrom_list)

    def set_fitness_value_list(self, chrom_list):
        fitness = []
        for chrom in chrom_list:
            fitness.append(chrom.get_fitness())

        #print(fitness)
        return(fitness)

    def get_chrom_list(self):
        return self.chrom_list
    
    def get_dimension():
        return Generation.num_dimensions

    def get_min_value_per_point():
        return Generation.min_value_per_point
    
    def get_max_value_per_point():
        return Generation.max_value_per_point

    def get_fitness_values_list(self):
        return self.generation_fitness_list



    def gen_first_generation(self, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point):
        #num_points_per_polytope = 5
        #num_chroms_per_Generation = 10
        #num_dimensions = num_dimensions

        polytope_points = []
        generation_polytopes = []

        for j in range(0, num_chroms_per_Generation):
            #chromosome_test_points=[]
            polytope_points = []
            for i in range(0,num_points_per_polytope):
                polytope_points.append(random.sample(range(min_value_per_point, max_value_per_point), num_dimensions))
            chromosome_polytope_points = np.array(polytope_points)
            #print(polytope_points)
            chromosom_test = Chromosom(chromosome_polytope_points)

            generation_polytopes.append(chromosom_test)
        
        generation_polytopes.sort(key=lambda x: x.fitness, reverse=True)
        #print(generation_polytopes)
        return generation_polytopes

class Chromosom:
    chrom_count = 0
    reflexive_politopes = {}

    def __init__(self, points):
        

        #print(Generation.num_dimensions)
        result = None
        while result is None:
            try:
                internal_points = []
                #print("###############################################")
                Chromosom.chrom_count = Chromosom.chrom_count + 1
                #print(Chromosom.chrom_count)
                
                self.gen_list = points
                self.q_hull = qhull(self.gen_list)
                self.A, self.b_half_space = compute_polytope_halfspaces(self.gen_list)

                self.num_stricktly_integral_points = self.gen_strictly_integral_points(self.q_hull)

                self.fitness = self.calc_fitness(self.num_stricktly_integral_points, self.b_half_space)
                #print(fitness)
                result = 1
            except:
                #print(points)
                #print(Generation.dimensions)
                
                polytope_points = []
                for i in range(0,Generation.num_points_per_polytope):
                    polytope_points.append(random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions))
                chromosome_polytope_points = np.array(polytope_points)
                
                points = chromosome_polytope_points
                #print(points)
                #print("try init chrom")
                #pass

        internal_points = []
        #print("###############################################")
        Chromosom.chrom_count = Chromosom.chrom_count + 1
        #print(Chromosom.chrom_count)
        
        self.gen_list = points
        self.q_hull = qhull(self.gen_list)
        self.A, self.b_half_space = compute_polytope_halfspaces(self.gen_list)

        self.num_stricktly_integral_points = self.gen_strictly_integral_points(self.q_hull)

        self.fitness = self.calc_fitness(self.num_stricktly_integral_points, self.b_half_space)

        if self.fitness == 0.0: 
            print(True)
            polytope_volume = self.q_hull.volume
            Chromosom.reflexive_politopes[polytope_volume] = convex_hull(self.gen_list)


        #print(fitness)

    # def get_q_hull(self):
    #     return self.q_hull

    # def get_b_half_space(self):
    #     return self. b_half_space
    
    def get_gen_list(self):
        return self.gen_list
    
    def get_fitness(self):
        return self.fitness

    def get_gen_list(self):
        return self.gen_list

    def grid_region(self, polyreg, res=None):
        # grid corners
        bbox = polyreg.bounding_box
        # grid resolution
        if res is None:
            density = 8
            res = [
                math.ceil(density * (b - a))
                for a, b in zip(*bbox)]
        if len(res) != polyreg.dim:
            raise ValueError((
                "`len(res)` must equal the polytope's dimension "
                "(which is {dim}), but instead `res` is:  {res}"
                ).format(dim=polyreg.dim, res=res))
        if any(n < 1 for n in res):
            raise ValueError((
                '`res` must contain `int` values >= 1, '
                'instead `res` equals:  {res}'
                ).format(res=res))
        linspaces = list()
        for a, b, n in zip(*bbox, res):
            r = np.linspace(a, b, num=n)
            linspaces.append(r)
        points = np.meshgrid(*linspaces)
        x = np.vstack(list(map(np.ravel, points)))
        x = x[:, polyreg.contains(x, abs_tol=0)]
        return (x, res)

    def enumerate_stricktly_integral_points(self, poly):
        a, b = poly.bounding_box
        a_int = np.floor(a)
        b_int = np.ceil(b)
        intervals = list(zip(a_int.flatten(), b_int.flatten()))
        box = box2poly(intervals)
        res = [int(b - a + 1) for a, b in intervals]
        grid, _ = self.grid_region(box, res=res)
        inside = poly.contains(grid, abs_tol=0)
        return grid[:, inside]

    def gen_strictly_integral_points(self, q_hull):
        integral_points = enumerate_integral_points(q_hull)
        integral_points = integral_points.transpose()
        integral_points = integral_points.astype(int)

        stricktly_integral_points = self.enumerate_stricktly_integral_points(q_hull)
        stricktly_integral_points = stricktly_integral_points.transpose()
        stricktly_integral_points = stricktly_integral_points.astype(int)
        num_stricktly_integral_points = len(stricktly_integral_points)
        return num_stricktly_integral_points

    def calc_fitness(self, ip_count, distances):
        result = 0
        if ip_count > 1:
            result -= 1
        for d in distances:
            result -= abs(d-1)
        
        if result > 0:
            result = result*(-1)

        #print(result)
        return result  
    
    def apply_crossover(chromosom_1, chromosom_2, crossover_type):
        
        if chromosom_1.get_fitness() > chromosom_2.get_fitness():
            chromosom_1_points = chromosom_1.get_gen_list()
            chromosom_2_points = chromosom_2.get_gen_list()
        else:
            chromosom_2_points = chromosom_1.get_gen_list()
            chromosom_1_points = chromosom_2.get_gen_list()
        
        crossover_type = crossover_type

        new_chromosom_gen_list = []
        if crossover_type == 1:
            
            if len(chromosom_1_points)>len(chromosom_2_points):
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
            else:
                length_new_chromosom = len(chromosom_2_points)
                random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

            #new_gen = []
            for i in range(0,length_new_chromosom):
                if i < random_crossover_point:
                    new_gen = []
                    for j in chromosom_1_points[i]:
                        new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])
                else:
                    new_gen = []
                    for j in chromosom_2_points[i]:
                        new_gen.append(j)

                new_chromosom_gen_list.append(new_gen)

        

        try:
            new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
        except:
            result = None
            while result is None:
                try:
                    #print(new_chromosom_gen_list)
                    #new_chromosom_gen_list = random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions)
                    
                    polytope_points = []
                    for i in range(0,Generation.num_points_per_polytope):
                        polytope_points.append(random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions))
                    chromosome_polytope_points = np.array(polytope_points)
                    
                    new_chromosom_gen_list = chromosome_polytope_points
                    new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
                    #print("try")
                    result = 1
                except:
                    #print("apply_crossover")
                    pass

        # print(new_chromosom_gen_list)
        # new_chromosom_gen_list = np.array(random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions))
        
        # while len(ConvexHull(new_chromosom_gen_list)) > Generation.num_dimensions:
        #     new_chromosom_gen_list = np.array(random.sample(range(Generation.min_value_per_point, Generation.max_value_per_point), Generation.num_dimensions))

        # new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
                
        return new_chromosom


In [12]:

# test_generation = Generation(None, 5, 10, 3, -3, 3)

# test_polytopes = test_generation.get_chrom_list()


# for i in test_polytopes:
#     print(i.get_fitness())
#     print(i.get_gen_list())


    

In [13]:
# Population Test:

# test_population = Population(number_of_generations, survival_rate, crossover_type, crossover_parent_choosing_type, chrom_list, num_points_per_polytope, num_chroms_per_Generation, num_dimensions, min_value_per_point, max_value_per_point)
test_population = Population(100, 0, 1, 0, None, 5, 50, 3, -3, 3)



True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [14]:
test_population_generations = test_population.get_all_generations()
test_population_generations_chroms = test_population_generations[1].get_chrom_list()
#print(test_population_generations_chroms)

all_fitness = []
for generation in test_population.get_all_generations():
    #fitness = [generation.get_fitness_values_list()]
    # for chrom in generation.get_chrom_list():
    #     fitness.append(chrom.get_fitness())

    print(generation.get_fitness_values_list())

    all_fitness.append(generation.get_fitness_values_list())
    #print(generation.get_fitness_values_list())
    #all_fitness.append(fitness)
#print(all_fitness)
print(len(all_fitness))
#print(all_fitness)

[-3.0, -4.0, -4.000000000000002, -6.0, -7.0, -8.0, -9.5, -10.0, -10.266666666666666, -10.333333333333332, -10.833333333333336, -11.0, -11.0, -13.999999999999998, -14.5, -15.1, -15.999999999999996, -18.0, -18.8, -22.0, -23.999999999999968, -24.0, -26.500000000000004, -26.73333333333333, -27.16666666666667, -27.25, -27.533333333333335, -28.0, -28.0, -28.333333333333332, -30.099999999999994, -30.33333333333336, -31.999999999999936, -33.0, -34.125, -34.53333333333332, -36.0, -36.5, -36.999999999999964, -37.0, -37.857142857142854, -39.16666666666661, -40.60000000000001, -42.000000000000014, -43.33333333333334, -43.99999999999999, -46.09761904761905, -52.999999999999964, -53.0, -61.1]
[-3.0, -3.333333333333333, -3.333333333333333, -4.0, -4.0, -4.000000000000002, -6.0, -7.0, -8.0, -8.5, -9.5, -10.0, -10.266666666666666, -10.333333333333332, -10.833333333333336, -11.0, -11.0, -11.333333333333332, -12.166666666666666, -13.0, -13.999999999999998, -14.5, -15.0, -15.1, -15.666666666666664, -15.999

In [15]:
# a = [1,2,3]
# b = a.copy()
# #b = []
# #b.append(a[0])
# #b.append(a[1])
# a[1] = 5

# print(a)
# print(b)

print(Chromosom.reflexive_politopes)
print(len(Chromosom.reflexive_politopes))

{2.4959999999999996: [array([ 1, -1,  0]), array([1, 0, 2]), array([-3,  2,  0]), array([ 1,  0, -2])], 2.144000000000001: [array([ 1, -1,  0]), array([1, 0, 2]), array([-1,  1,  0]), array([-3,  2,  0]), array([ 1, -1, -2])], 2.0960000000000014: [array([ 1, -1,  0]), array([1, 0, 2]), array([-1,  1,  0]), array([-3,  2,  0]), array([ 1, -1, -2])], 2.288000000000001: [array([ 1, -1,  0]), array([1, 0, 2]), array([-1,  1,  0]), array([-3,  2,  0]), array([ 1, -1, -2])], 2.096000000000001: [array([ 1, -1,  0]), array([1, 0, 2]), array([-1,  1,  0]), array([-3,  2,  0]), array([ 1, -1, -2])], 2.048000000000001: [array([ 1, -1,  0]), array([1, 0, 2]), array([-1,  1,  0]), array([-3,  2,  0]), array([ 1, -1, -2])], 2.1120000000000014: [array([ 1, -1,  0]), array([1, 0, 2]), array([-1,  1,  0]), array([-3,  2,  0]), array([ 1, -1, -2])], 1.856000000000001: [array([ 1, -1,  0]), array([1, 0, 2]), array([-1,  1,  0]), array([-3,  2,  0]), array([ 1, -1, -2])], 2.7999999999999994: [array([ 1, -

In [16]:
# Crossover test - to be checked

# # #def apply_crossover(chromosom_1, chromosom_2, crossover_type):
# chromosom_1_points = test_polytopes[1].get_gen_list()
# chromosom_2_points = test_polytopes[0].get_gen_list()
# # crossover_type = 1

# # new_chromosom_gen_list = []
# # if crossover_type == 1:
    

# #     if len(chromosom_1_points)>len(chromosom_2_points):
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)
# #     else:
# #         length_new_chromosom = len(chromosom_2_points)
# #         random_crossover_point = random.randint(1,len(chromosom_2_points)-1)

# #     #new_gen = []
# #     for i in range(0,length_new_chromosom):
# #         if i <= random_crossover_point:
# #             new_gen = []
# #             for j in chromosom_1_points[i]:
# #                 new_gen.append(j)#[chromosom_1_points[0],chromosom_1_points[1]])
# #         else:
# #             new_gen = []
# #             for j in chromosom_2_points[i]:
# #                 new_gen.append(j)

# #         new_chromosom_gen_list.append(new_gen)

# # new_chromosom = Chromosom(np.array(new_chromosom_gen_list))
# #return new_chromosom

# new_chromosom = Chromosom.apply_crossover(test_polytopes[1], test_polytopes[0], 1)

# #print(random_crossover_point)
# print(chromosom_1_points)
# print(chromosom_2_points)
# print(new_chromosom.get_gen_list())
# print(new_chromosom.get_fitness())


In [17]:
# import numpy as np
# import polytope

# # halfspace representation computed using `polytope`
# # from the vertex representation given in the question
# vertices = np.array([[0, -1], [-1, 0], [-1, 2], [1, 0], [1, -1]])
# poly = polytope.qhull(vertices)
# # first halfspace representation
# A = np.array([
#     [0, -1],
#     [-1, 0],
#     [-1, -1],
#     [1, 1],
#     [1, 0]])
# b = np.array([1, 1, 1, 1, 1])
# question_poly_1 = polytope.Polytope(A, b)
# # second halfspace representation
# A = np.array([
#     [-0.70711, -0.70711],
#     [-1, -0],
#     [0.70711, 0.70711],
#     [0, -1],
#     [1, 0]])
# b = np.array([0.70711, 1, 0.70711, 1, 1])

# question_poly_2 = polytope.Polytope(A, b)
# # check that all the above halfspace representations
# # represent the same polytope
# assert poly == question_poly_1, (poly, question_poly_1)
# assert poly == question_poly_2, (poly, question_poly_2)

In [18]:
# inner_point = np.array([0,0]) # in this case it is the origin
# fac1 = polytope.quickhull.Facet([[0, -1], [-1, 0]])
# fac2 = polytope.quickhull.Facet([[-1, 2], [1, 0]])
# fac3 = polytope.quickhull.Facet([[1, -1], [0, -1]])

# d1 = polytope.quickhull.distance(inner_point, fac1)
# d2 = polytope.quickhull.distance(inner_point, fac2)
# d3 = polytope.quickhull.distance(inner_point, fac3)
# print(d1)
# print(d2)
# print(d3)

Let's visualize our 2D polytope. For this purpose we use the [Polytope Library](https://github.com/tulip-control/polytope)

In [19]:
# project into 2D



In [20]:

# print(polytope.polytope.cheby_ball(P))

# min_V = np.min(V, axis=0) #min val of each cols
# min_x = min_V[0]
# min_y = min_V[1]
# max_V = np.max(V, axis=0) #max val of each cols
# max_x = max_V[0]
# max_y = max_V[1]

# ax = P.plot(linestyle="solid", linewidth=1, color="lightblue", edgecolor="red")
# ax.set_xlim([min_x-1, max_x+1])
# ax.set_ylim([min_y-1, max_y+1])
# plt.show()